In [ ]:
import requests
import pandas as pd
import re 

## FUNCTION TO GET THE LAST PAGE
def get_last_page(url):       
    r = requests.get(url)    
    url = pd.DataFrame(r.json()['links'])      
    try: 
        last_page = url['href'][3]    
        last_page_number = re.findall('[0-9]+', last_page) 
        last_page_number = last_page_number[3]   
    except:
        last_page_number = 1
        print('No last page')
    return last_page_number

## API URL to get deputies ID
url = 'https://dadosabertos.camara.leg.br/api/v2/deputados?ordem=ASC&ordenarPor=nome'
request = requests.get(url)
df_dep_info = pd.DataFrame(request.json()['dados'])

## lists to store the data and the ids
list = []
id_list = []
name_list = []
partido_list = []
uf_list = []
urlFoto_list = []
email_list = []

for year in range(2020, 2022+1):
    page = 1

    ## counter to select index of rows
    count = 0

    ## request the data for each id
    for id in df_dep_info['id']:          
        url = 'https://dadosabertos.camara.leg.br/api/v2/deputados/'+ str(id) + '/despesas?ano=' + str(year) + '&pagina=' + str(page) + '&itens=100&ordem=ASC&ordenarPor=ano'   
        print(id)
        last_page = get_last_page(url) 
        count += 1  
        

    ## Pagination
        for page in range(1, int(last_page)+1):    
            url = 'https://dadosabertos.camara.leg.br/api/v2/deputados/'+ str(id) + '/despesas?ano=' + str(year) + '&pagina=' + str(page) + '&itens=100&ordem=ASC&ordenarPor=ano' 
            r2 = requests.get(url)
            df_expenses = pd.DataFrame(r2.json()['dados'])        
            list.append(df_expenses)

            ## counting how many rows appears
            count_rows = len(r2.json()['dados'])

            for i in range(count_rows):
                id_list.append(id)
                name_list.append(df_dep_info['nome'][count-1])
                partido_list.append(df_dep_info['siglaPartido'][count-1])
                uf_list.append(df_dep_info['siglaUf'][count-1])
                urlFoto_list.append(df_dep_info['urlFoto'][count-1])
                email_list.append(df_dep_info['email'][count-1])
        

## The line below will allow us to transform our list into a dataframe
df_expenses = pd.concat(list)

## add the columns information to the dataframe
df_expenses['id'] = id_list
df_expenses['nome'] = name_list
df_expenses['siglaPartido'] = partido_list
df_expenses['siglaUf'] = uf_list
df_expenses['urlFoto'] = urlFoto_list
df_expenses['email'] = email_list

## Organize the dataframe
df_expenses = df_expenses[['ano','mes', 'id', 'nome', 'siglaPartido', 'siglaUf', 'urlFoto', 'email', 'tipoDespesa', 'dataDocumento', 'valorDocumento', 'urlDocumento', 'nomeFornecedor', 'cnpjCpfFornecedor']].copy()

## Save the dataframe in a csv file
df_expenses.to_csv('expenses.csv', index=False)

In [9]:
# Library to connect to Oracle
import cx_Oracle

# patch to instantclient_21_3
#cx_Oracle.init_oracle_client(lib_dir=r"C:\\oracle\\instantclient_21_3")


# Variables to connect to Oracle
username = "ADMIN"
password = "Q1w2e3r4t5y6u7i8o9p0"
dsn = '(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.sa-vinhedo-1.oraclecloud.com))(connect_data=(service_name=g19cc7d208b76c0_adbarticle01_high.adb.oraclecloud.com))(security=(ssl_server_cert_dn="CN=adb.sa-vinhedo-1.oraclecloud.com, O=Oracle Corporation, L=Redwood City, ST=California, C=US")))'

# Connect to Oracle
connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)

# Drop table if it exists
cursor.execute("""begin
			        execute immediate 'drop table data_tab';
			        exception when others then if sqlcode <> -942 then raise; end if;
		            end;""")

#Create table
cursor.execute("""create table data_tab(
                                        ano INT, 
                                        mes INT, 
                                        id INT, 
                                        nome VARCHAR(255), 
                                        siglaPartido VARCHAR(255), 
                                        siglaUf VARCHAR(5), 
                                        urlFoto VARCHAR(255), 
                                        email VARCHAR(255), 
                                        tipoDespesa VARCHAR(255), 
                                        dataDocumento VARCHAR(255), 
                                        valorDocumento INT, 
                                        urlDocumento VARCHAR(255), 
                                        nomeFornecedor VARCHAR(255), 
                                        cnpjCpfFornecedor VARCHAR(255))""")

connection.commit()


In [7]:
import pandas as pd

main = pd.read_csv('expenses.csv' , sep=',')

In [ ]:
# library to insert data to database
from sqlalchemy import create_engine
import pandas as pd

# import data from csv
main = pd.read_csv('expenses.csv' , sep=',')

# Variables to connect to Oracle
username = "ADMIN"
password="topsecret123"
dsn = '(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.sa-vinhedo-1.oraclecloud.com))(connect_data=(service_name=g19cc7d208b76c0_adbarticle01_high.adb.oraclecloud.com))(security=(ssl_server_cert_dn="CN=adb.sa-vinhedo-1.oraclecloud.com, O=Oracle Corporation, L=Redwood City, ST=California, C=US")))'

# create engine
engine = create_engine(f'oracle://{username}:{password}@{dsn}/?encoding=UTF-8&nencoding=UTF-8', max_identifier_length=128)

# insert dataframe into oracle database 
main.to_sql('data_tab', con = engine, if_exists = 'append', chunksize=1000) 
      

In [ ]:
import cx_Oracle
from sqlalchemy import create_engine
import pandas as pd


#if sys.platform.startswith("darwin"):
  #  cx_Oracle.init_oracle_client(
      #  lib_dir=os.environ.get("HOME")+"/Downloads/instantclient_19_8",
      #  config_dir="")
#elif sys.platform.startswith("win"):
    #cx_Oracle.init_oracle_client(lib_dir=r"C:\\oracle\\instantclient_21_3")
# else assume system library search path includes Oracle Client libraries
# On Linux, use ldconfig or set LD_LIBRARY_PATH, as described in installation documentation.

username = "ADMIN"
# set the password in an environment variable called "MYPW" for security
password="Q1w2e3r4t5y6u7i8o9P0"
vardsn = '(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.sa-vinhedo-1.oraclecloud.com))(connect_data=(service_name=g19cc7d208b76c0_db20220520101033_high.adb.oraclecloud.com))(security=(ssl_server_cert_dn="CN=adb.sa-vinhedo-1.oraclecloud.com, O=Oracle Corporation, L=Redwood City, ST=California, C=US")))'


engine = create_engine(f'oracle://{username}:{password}@{vardsn}/?encoding=UTF-8&nencoding=UTF-8', max_identifier_length=128)
print('criado o eng')
main.to_sql('data_tab', con = engine, if_exists = 'append', chunksize=1000) 